# Project: Data Cleaning Challenge

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata


# display outputs inline
%matplotlib inline

# to see all columns and rows to using the display max column parameter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# set the display colwidth for seeing the entire text contents
pd.set_option('display.max_colwidth', -1)

C:\Users\USER\AppData\Local\Temp\ipykernel_22476\2121266886.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
# Importing data
fifa = pd.read_csv('fifa21 raw data v2.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_22476\3160915589.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa = pd.read_csv('fifa21 raw data v2.csv')


In [3]:
# make a copy of the data so as not to tamper with the original copy
fifa_clean = fifa.copy()

# Data Inspection

In [4]:
# view a sample of the data
fifa_clean.sample(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
5628,193851,D. Blacha,David Blacha,https://cdn.sofifa.com/players/193/851/21_60.png,http://sofifa.com/player/193851/david-blacha/210006/,Poland,29,69,69,\n\n\n\nVfL Osnabrück,2018 ~ 2021,"CM, CDM, CAM",174cm,72kg,Right,69,CM,"Jul 1, 2018",NaN,€1.4M,€7K,€1.6M,305,64,57,51,76,57,331,68,64,62,64,73,376,72,65,84,72,83,343,62,75,76,63,67,302,57,69,49,68,59,78,171,65,59,47,51,11,7,11,7,15,1879,396,4 ★,3★,Medium,Medium,1 ★,68,60,69,72,61,66,3
11870,242938,G. Ayadi,Ghazi Ayadi,https://cdn.sofifa.com/players/242/938/21_60.png,http://sofifa.com/player/242938/ghazi-ayadi/210006/,Tunisia,23,64,71,\n\n\n\nDamac FC,2020 ~ 2023,"CM, CDM",179cm,75kg,Right,67,CDM,"Mar 6, 2020",NaN,€1.2M,€5K,€1.2M,293,53,59,61,66,54,303,64,47,61,65,66,340,71,68,65,66,70,342,64,63,77,71,67,315,71,65,55,58,66,56,183,57,65,61,52,12,10,7,13,10,1828,390,3 ★,3★,Medium,Medium,1 ★,69,62,60,65,62,72,1
7780,236390,Pol Valentín,Pol Valentín Sancho,https://cdn.sofifa.com/players/236/390/21_60.png,http://sofifa.com/player/236390/pol-valentin-sancho/210006/,Spain,23,67,74,\n\n\n\nCF Fuenlabrada,2020 ~ 2022,"RB, RWB, RM",180cm,73kg,Right,67,RB,"Aug 31, 2020",NaN,€1.9M,€2K,€1.7M,229,63,30,48,59,29,267,65,61,21,57,63,398,91,91,84,62,70,288,55,63,71,61,38,271,60,59,66,54,32,59,183,61,62,60,62,15,8,13,12,14,1698,375,3 ★,3★,High,Medium,1 ★,91,38,57,66,60,63,13
7357,251677,Bernardo Martins,Bernardo Brites Martins,https://cdn.sofifa.com/players/251/677/21_60.png,http://sofifa.com/player/251677/bernardo-brites-martins/210006/,Portugal,22,67,76,\n\n\n\nFC Paços de Ferreira,2019 ~ 2022,"LW, RW",176cm,70kg,Left,69,CAM,"Jul 9, 2019",NaN,€2.3M,€3K,€2.8M,281,65,57,35,68,56,343,70,70,67,65,71,345,69,69,72,64,71,303,70,56,59,62,56,262,43,25,66,65,63,60,92,38,29,25,49,13,5,11,6,14,1675,353,3 ★,3★,Medium,Medium,1 ★,69,60,66,70,31,57,4
7548,236344,J. Molumby,Jayson Molumby,https://cdn.sofifa.com/players/236/344/21_60.png,http://sofifa.com/player/236344/jayson-molumby/210006/,Republic of Ireland,20,67,79,\n\n\n\nBrighton & Hove Albion,2016 ~ 2023,"CM, CDM",178cm,64kg,Right,69,CM,"Jul 1, 2016",NaN,€2.4M,€13K,€2.7M,263,49,48,48,73,45,296,66,46,42,72,70,324,65,63,62,69,65,275,54,43,71,56,51,312,74,64,57,60,57,64,178,57,63,58,46,8,7,10,12,9,1694,367,3 ★,2★,Medium,Medium,1 ★,64,51,63,67,59,63,31


#### Observations
1. Name of clubs are messy, they contain irrelevant characters
2. The Contract column values dont follow the same format
3. The Height column values arent well formatted so would likely give rise to wrong datatype, this is observed in the weight column too.
4. The weight column is graduated in the wrong units, it ought to be in lbs
5. Loan end date looks to contain lots of NAN values.
6. The Value, Wage and Release Clause columns arent well formatted, the currency type should be in dollars and some conversions should be made.
7. The W/F, SM and IR columns are not well formatted and would likely give rise to the wrong datatype
8. Some Column Names arent descriptive enough.
9. There are inconsistencies in some values, like in Hits column where some data have a K in them.

In [5]:
fifa_clean.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/210006/,Argentina,33,93,93,\n\n\n\nFC Barcelona,2004 ~ 2021,"RW, ST, CF",170cm,72kg,Left,93,RW,"Jul 1, 2004",NaN,€103.5M,€560K,€138.4M,429,85,95,70,91,88,470,96,93,94,91,96,451,91,80,91,94,95,389,86,68,72,69,94,347,44,40,93,95,75,96,91,32,35,24,54,6,11,15,14,8,2231,466,4 ★,4★,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/210006/,Portugal,35,92,92,\n\n\n\nJuventus,2018 ~ 2022,"ST, LW",187cm,83kg,Right,92,ST,"Jul 10, 2018",NaN,€63M,€220K,€75.9M,437,84,95,90,82,86,414,88,81,76,77,92,431,87,91,87,95,71,444,94,95,84,78,93,353,63,29,95,82,84,95,84,28,32,24,58,7,11,15,14,11,2221,464,4 ★,5★,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,2014 ~ 2023,GK,188cm,87kg,Right,91,GK,"Jul 16, 2014",NaN,€120M,€125K,€159.4M,95,13,11,15,43,13,109,12,13,14,40,30,307,43,60,67,88,49,268,59,78,41,78,12,140,34,19,11,65,11,68,57,27,12,18,437,87,92,78,90,90,1413,489,3 ★,1★,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyne/210006/,Belgium,29,91,91,\n\n\n\nManchester City,2015 ~ 2023,"CAM, CM",181cm,70kg,Right,91,CAM,"Aug 30, 2015",NaN,€129M,€370K,€161M,407,94,82,55,94,82,441,88,85,83,93,92,398,77,76,78,91,76,408,91,63,89,74,91,408,76,66,88,94,84,91,186,68,65,53,56,15,13,5,10,13,2304,485,5 ★,4★,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silva-santos-jr/210006/,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,2017 ~ 2022,"LW, CAM",175cm,68kg,Right,91,LW,"Aug 3, 2017",NaN,€132M,€270K,€166.5M,408,85,87,62,87,87,448,95,88,89,81,95,453,94,89,96,91,83,357,80,62,81,50,84,356,51,36,87,90,92,93,94,35,30,29,59,9,9,15,15,11,2175,451,5 ★,5★,High,Medium,5 ★,91,85,86,94,36,59,595


In [6]:
# Checking for duplicates
fifa_clean[fifa_clean.duplicated()]

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,Contract,Positions,Height,Weight,Preferred Foot,BOV,Best Position,Joined,Loan Date End,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits


#### No duplicate data in the dataset


In [7]:
# Identifying Null Values
null_counts = fifa_clean.isnull().sum()  # Count the number of null values in each column
null_cols = null_counts[null_counts > 0].index.tolist()  # Get the column names with null values

# Create a new dataframe with only the columns with null values and their corresponding null counts
null_fifa = pd.DataFrame({'Columns': null_cols, 'Null Count': null_counts[null_cols].tolist()})

null_fifa

,Columns,Null Count
0,Loan Date End,17966
1,Hits,2595


#### Observations
1. Only two columns contain Null Values, the Loan Date End and the Hits column, the Loan Date End Nulls seem to be significant.

In [8]:
# checking the datatypes and other information for each columns for each column
fifa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

#### Observation
1. There are some inappropriate data type assigned ro some columns

In [9]:
# view summary statistics of the fifa data
fifa_clean.describe()

,ID,Age,↓OVA,POT,BOV,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY
count,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.00000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000,18979.000000
mean,226403.384794,25.194109,65.718636,71.136414,66.751726,248.938142,49.688392,45.842405,51.942726,58.768112,42.696507,256.479214,55.587491,47.246957,42.391011,52.725381,58.528373,317.718689,64.357553,64.412930,63.366721,61.655619,63.925865,296.605195,57.801676,64.577375,62.653986,64.786922,46.785236,254.199747,55.591285,46.394489,50.298962,53.857105,48.057906,58.07366,139.909268,46.559197,47.705622,45.644449,81.436693,16.416302,16.207124,16.076453,16.217187,16.519627,1595.286949,355.702197,67.453975,53.457031,57.681016,62.875020,49.866221,64.368934
std,27141.054157,4.710520,6.968999,6.114635,6.747193,74.299428,18.131153,19.567081,17.294409,14.519106,17.646937,78.650601,18.761314,18.207790,17.227947,15.178151,16.565892,55.879046,14.889981,14.638743,14.596277,9.072114,14.062285,50.732390,13.308747,11.844870,15.804223,12.488672,19.300534,64.595613,17.140414,20.698078,19.428701,13.708574,15.655999,12.07911,61.219472,20.139324,21.367690,20.922087,84.665579,17.554194,16.816305,16.491103,17.002239,17.854079,269.874789,40.761117,10.677859,13.827425,10.081857,9.927415,16.443213,9.601883
min,41.000000,16.000000,47.000000,47.000000,48.000000,42.000000,6.000000,3.000000,5.000000,7.000000,3.000000,40.000000,5.000000,4.000000,5.000000,5.000000,5.000000,122.000000,13.000000,12.000000,14.000000,24.000000,12.000000,122.000000,18.000000,15.000000,12.000000,16.000000,4.000000,50.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.00000,20.000000,3.000000,5.000000,4.000000,10.000000,2.000000,2.000000,2.000000,2.000000,2.000000,747.000000,232.000000,25.000000,16.000000,25.000000,25.000000,12.000000,28.000000
25%,210135.000000,21.000000,61.000000,67.000000,62.000000,222.000000,38.000000,30.000000,44.000000,54.000000,30.000000,222.000000,49.000000,35.000000,31.000000,43.000000,54.000000,289.000000,57.000000,57.000000,55.000000,56.000000,56.000000,264.000000,48.000000,58.000000,55.000000,57.000000,32.000000,227.500000,44.000000,25.000000,40.000000,45.000000,39.000000,50.00000,83.000000,29.000000,27.000000,25.000000,48.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1452.000000,327.000000,61.000000,44.000000,51.000000,57.000000,35.000000,58.000000
50%,232418.000000,25.000000,66.000000,71.000000,67.000000,263.000000,54.000000,49.000000,55.000000,62.000000,44.000000,269.000000,61.000000,49.000000,41.000000,56.000000,63.000000,327.000000,67.000000,67.000000,66.000000,62.000000,66.000000,302.000000,59.000000,65.000000,66.000000,66.000000,51.000000,263.000000,58.000000,53.000000,55.000000,55.000000,49.000000,59.00000,159.000000,52.000000,55.000000,52.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1627.000000,356.000000,68.000000,56.000000,58.000000,64.000000,53.000000,65.000000
75%,246922.500000,29.000000,70.000000,75.000000,71.000000,297.

# Data Cleaning

### Handling inconsistencies in the data values

#### Rename Columns

In [10]:
fifa_clean.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [11]:
# Give columns more descriptive names for easy understanding, Abbreviations can be ambiguous and can lead to misunderstandings when analyzing the data
fifa_clean.rename(columns = {'LongName':'Full Name','Age':'Age(2021)', '↓OVA':'Overall Rating', 'POT':'Potential Rating',
                             'Height':'Height(ft)', 'Weight':'Weight(lbs)', 'BOV':'Best Overall Rating',
                            'Value':'Value(€)', 'Wage':'Wage(€)', 'ReleaseClause':'Release Clause(€)',
                            'W/F':'Weaker Foot(★)', 'SM':'Skils(★)', 'A/W':'Attacking Contribution',
                             'D/W':'Defensive Contribution', 'IR':'Injury Resistance(★)', 'PAC':'Pace', 'SHO':'Shot', 
                             'PAS':'Pass Accuracy', 'DRI':'Dribbling Ability',
                            'DEF':'Defensive Ability', 'PHY': 'Physicality', 'Hits': 'Profile Views'}, inplace = True)

fifa_clean.columns

Index(['ID', 'Name', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality',
       'Age(2021)', 'Overall Rating', 'Potential Rating', 'Club', 'Contract',
       'Positions', 'Height(ft)', 'Weight(lbs)', 'Preferred Foot',
       'Best Overall Rating', 'Best Position', 'Joined', 'Loan Date End',
       'Value(€)', 'Wage(€)', 'Release Clause', 'Attacking', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill',
       'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control',
       'Movement', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
       'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
       'Long Shots', 'Mentality', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defending', 'Marking',
       'Standing Tackle', 'Sliding Tackle', 'Goalkeeping', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       'Total Stats', 'Base Stats', 'Weaker

#### Datatype Conversion

In [12]:
fifa_clean['ID'] = fifa_clean['ID'].astype(str)
fifa_clean['Preferred Foot'] = fifa_clean['Preferred Foot'].astype('category')
fifa_clean['Joined'] = pd.to_datetime(fifa_clean['Joined'])
fifa_clean['Loan Date End'] = pd.to_datetime(fifa_clean['Loan Date End'])
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].astype('category')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].astype('category')
fifa_clean['Attacking Contribution'] = fifa_clean['Attacking Contribution'].astype('category')
fifa_clean['Defensive Contribution'] = fifa_clean['Defensive Contribution'].astype('category')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].astype('category')

#### Cleaning the Profile Views Column

This column has 2595 missing  values, values of over 1000 are represented with a decimal and a k, handling these issue would make the column cleaner, To fill the missing values, the mode would be used as the data is skewed

In [13]:
# looking at the Profile Views column to observe inconsistencies
fifa_clean['Profile Views'].loc[6:8]

6    246 
7    120 
8    1.6K
Name: Profile Views, dtype: object

In [14]:
# Replace 'K' and 'M' with their corresponding numeric values in the 'Profile Views' column
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].str.replace('K', 'e3').str.replace('M', 'e6')

# Convert the 'Profile Views' column to numeric type
fifa_clean['Profile Views'] = pd.to_numeric(fifa_clean['Profile Views'])

# Filter rows where the 'Profile Views' column contains 'K'
k_values = fifa_clean[fifa_clean['Profile Views'].astype(str).str.contains('e3')]

fifa_clean['Profile Views'].loc[6:8]

6    246.0 
7    120.0 
8    1600.0
Name: Profile Views, dtype: float64

In [15]:
# check the skewness of the hits values to determine what measure of center is best to fill null values
fifa_clean['Profile Views'].skew()

31.04253072771258

In [16]:
# get the mode to fill in na with
mode_hits = fifa_clean['Profile Views'].mode()
mode_hits

0    1.0
Name: Profile Views, dtype: float64

In [17]:
# fill in missing values in Profile Views column
fifa_clean['Profile Views'].fillna(1.0, inplace=True)

# change profile views to int data type
fifa_clean['Profile Views'] = fifa_clean['Profile Views'].astype(int)

# Check to confirm there are no Null values
fifa_clean[fifa_clean['Profile Views'].isna()]

,ID,Name,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Contract,Positions,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Loan Date End,Value(€),Wage(€),Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views


####  Cleaning the Contract column

The Contract Column is having many discrepancies in value format, it also has some relationship with the Joined and loan date end column, from the contract column, a contract type column would be derived, this column tells us if the players contract is a permanent deal, a loan deal or a free agent(not under contract with any football club and is therefore free to sign with any club of their choosing). After which a contract start and contract end column is derived from the contract column, joined column and loan date column, these new columns tells when a player contract started, for a free agent this would be non applicable (N/A) as he has no contract while the contract end tells when the contracts would be terminated, for free agents the contract end would be N/A (not applicable) as a player without a contract cant have end date

In [18]:
fifa_clean['Contract'].loc[397:405]

397    Free                
398    2019 ~ 2023         
399    2017 ~ 2023         
400    2019 ~ 2024         
401    2014 ~ 2026         
402    2018 ~ 2023         
403    2019 ~ 2023         
404    2019 ~ 2024         
405    Jun 30, 2021 On Loan
Name: Contract, dtype: object

In [19]:
fifa_clean['Contract Type'] = fifa_clean['Contract'].apply(lambda x: 'Free' if x == 'Free' 
                                         else 'On Loan' if 'On Loan' in x 
                                         else 'Permanent' if '~' in x 
                                         else np.nan)


In [20]:
# define a function to parse the values in the 'Joined' and 'Loan Date End' columns
def parse_dates(row):
    if row['Contract'] == 'Free':
        start = np.nan
        end = np.nan
    elif 'On Loan' in row['Contract']:
        start = pd.to_datetime(row['Joined']).year
        end = row['Contract'].split(' ')[2]
    else:
        start = int(row['Contract'][:4])
        end = int(row['Contract'][-4:])
    return pd.Series({'Contract Start': start, 'Contract End': end})

# apply the function to create the new columns
fifa_clean[['Contract Start', 'Contract End']] = fifa_clean.apply(parse_dates, axis=1)


# convert float columns to object datatype and remove the unwanted characters
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype(str)
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype(str)

# Define a function to remove the '.i0' part from a string
def remove_dot_i(s):
    return s.split('.')[0]

# Apply the function to the fifa_clean['Contract Start']  and fifa_clean['Contract end'] column using .map()
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].map(remove_dot_i)
fifa_clean['Contract End'] = fifa_clean['Contract End'].map(remove_dot_i)

# Assigning the contract start and contract end columns into categories
fifa_clean['Contract Start'] = fifa_clean['Contract Start'].astype('category')
fifa_clean['Contract End'] = fifa_clean['Contract End'].astype('category')

# view samples of the result
fifa_clean['Contract'].loc[397:405]

397    Free                
398    2019 ~ 2023         
399    2017 ~ 2023         
400    2019 ~ 2024         
401    2014 ~ 2026         
402    2018 ~ 2023         
403    2019 ~ 2023         
404    2019 ~ 2024         
405    Jun 30, 2021 On Loan
Name: Contract, dtype: object

#### Cleaning the values in the Club, Name and Full Name columns

The club column values have messy and irrelevant characters in them, this would be cleaned. The Name, Full Name and Club colums have some diacritic characters whic could arise due to different languages and would affect alphabetic ordering during visualizations

In [21]:
# take a look at the column to see the format of naming
fifa_clean['Club'].sample(3)

5608     \n\n\n\nAl Fayha        
16066    \n\n\n\nDoncaster Rovers
6642     \n\n\n\nFC Midtjylland  
Name: Club, dtype: object

In [22]:
# Replacing the '\n' with an empty string
fifa_clean['Club'] = fifa_clean['Club'].str.replace('\n', '')
fifa_clean['Club'].sample(3)

3651    Celtic             
9329    Preston North End  
7325    Queens Park Rangers
Name: Club, dtype: object

In [23]:
# Examples of diacritic names
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Süle          
Full Name    Niklas Süle      
Club         FC Bayern München
Name: 110, dtype: object

In [24]:
# Define regular expression pattern to match special characters in Name, LongName and Club columns
def remove_diacritics(text):
    """
    This function removes diacritics (accent marks) from text
    """
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

# Apply remove_diacritics function to Name, LongName, and Club columns
fifa_clean['Name'] = fifa_clean['Name'].apply(remove_diacritics)
fifa_clean['Full Name'] = fifa_clean['Full Name'].apply(remove_diacritics)
fifa_clean['Club'] = fifa_clean['Club'].apply(remove_diacritics)

# Check if Diacritic Names were removed
fifa_clean[['Name','Full Name','Club']].loc[110]

Name         N. Sule          
Full Name    Niklas Sule      
Club         FC Bayern Munchen
Name: 110, dtype: object

### 

####  Cleaning the Height column
The height column has values with contrasting units, some are in cm and others in feet, to clean it the units are removed from the values and then they are all converted to ftin in decimals, a descriptive column name would ensure the values are easily understood. The ftin unit is a more appropriate unit here because the weight column is tobe formatted in lbs, with this if need be for creating a BMI column it wont need any more formatting

In [25]:
fifa_clean['Height(ft)'].sample(6)

16857    189cm
13818    187cm
9112     180cm
7651     194cm
14921    185cm
11846    186cm
Name: Height(ft), dtype: object

In [26]:
# define a function that converts cm to feets and inches, This is my preferred unit as the weight is in lbs, this makes calculations that demands precision like the BMI easy

def convert_cm_to_feet(cm):
    cm = float(cm.replace('cm', '')) # remove 'cm' units and convert to float
    feet = int(cm / 30.48) # calculate number of feet
    inches = round((cm / 2.54) % 12) # calculate number of inches and round to nearest integer
    return f"{feet}'{inches}\"" # return result in feet and inches format

# apply the conversion function only to values that are in cm
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(lambda x: convert_cm_to_feet(x) if 'cm' in x else x)

# view the result
fifa_clean['Height(ft)'].sample(5)

9359     6'3" 
1858     6'0" 
11118    5'10"
12563    5'9" 
11043    6'2" 
Name: Height(ft), dtype: object

In [27]:
# define a function to convert feet and inches format to decimal format
def convert_feet_to_dec(feet):
    feet_list = feet.split("'")
    ft = float(feet_list[0])
    if len(feet_list) > 1:
        inches = float(feet_list[1].replace('"', ''))
    else:
        inches = 0
    dec = ft + inches / 12
    return round(dec, 2) # round to 1 decimal place


# apply the conversion function to the 'Height' column
fifa_clean['Height(ft)'] = fifa_clean['Height(ft)'].apply(convert_feet_to_dec)

# view result
fifa_clean['Height(ft)'].sample(6)

6000     5.75
13807    5.75
4647     6.08
1829     6.08
17950    6.33
13507    5.67
Name: Height(ft), dtype: float64

#### Cleaning the Weight Column
The weight column has values with contrasting units, some are in lbs and others in kg, to clean it the units are removed from the values and then they are all converted to lbs, a descriptive column name would ensure the values are easily understood. The lbs unit is a more appropriate unit here because the height column is formatted in ftin, with this if need be for creating a BMI column it wont need any more formatting

In [28]:
fifa_clean['Weight(lbs)'].sample(6)

4742     84kg
2100     87kg
3994     83kg
3536     80kg
11171    74kg
8629     78kg
Name: Weight(lbs), dtype: object

In [29]:
# define a function that converts kg to lbs
def convert_kg_to_lbs(kg):
    kg = float(kg.replace('kg', '')) # remove 'kg' units and convert to float
    lbs = int(kg * 2.20462) # calculate kg
    return f"{lbs}lbs" #returns result in lbs format

# apply the conversion function only to values that are in kg
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].apply(lambda x: convert_kg_to_lbs(x) if 'kg' in x else x)

# remove the lbs units from the vale
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].str.replace('lbs', '')

# convert to float datatype
fifa_clean['Weight(lbs)'] = fifa_clean['Weight(lbs)'].astype(float)

# view result of code
fifa_clean['Weight(lbs)'].sample(3)

9865    165.0
9183    158.0
5765    163.0
Name: Weight(lbs), dtype: float64

#### Cleaning the Value, Wage and Release Clause columns
The Value, Wage and Release columns have similar mess issues, have currency symbols in them and are formatted in thousand and millions decimal points and have K and M attached to them. These symbols would be dropped and the thousands and millions format would be made uniform. The choice of of the euro(€) currency is because most transfer deals are negotiated in euros

In [30]:
fifa_clean[['Value(€)','Wage(€)', 'Release Clause']].head()

,Value(€),Wage(€),Release Clause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M


In [31]:
# Define a function to convert euro and k/m suffixes to USD and millions
def convert_value(val):
    if val.startswith('€'):
        val = val.replace('€', '')
        if val.endswith('K'):
            val = float(val.replace('K', '')) * 1000
        elif val.endswith('M'):
            val = float(val.replace('M', '')) * 1000000
    
    return pd.to_numeric(val)

# Apply the conversion function to the DataFrame
fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']] = fifa_clean[['Value(€)', 'Wage(€)', 'Release Clause']].applymap(convert_value)

# Set the display format of floats to include a comma delimiter
pd.options.display.float_format = '{:,.2f}'.format

fifa_clean.rename(columns={'Release Clause': 'Release Clause(€)'}, inplace=True)

# quick glance at the resulting data
fifa_clean[['Value(€)','Wage(€)', 'Release Clause(€)']].sample(5)

,Value(€),Wage(€),Release Clause(€)
11643,"1,400,000.00","7,000.00","1,500,000.00"
10915,"675,000.00","2,000.00","763,000.00"
14393,"60,000.00",500.00,"75,000.00"
8017,"900,000.00","4,000.00","1,100,000.00"
10190,0.00,0.00,0.00


#### Cleaning columns with star ratings (Weaker Foot, Skills, Injury Resistance)
The columns with values as star ratings have symbols to represent the number of stars received for each category by a player, these star symbols would be dropped

In [32]:
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].head()

,Weaker Foot(★),Skils(★),Injury Resistance(★)
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★


In [33]:
fifa_clean['Weaker Foot(★)'] = fifa_clean['Weaker Foot(★)'].str.replace('\u2605', '')
fifa_clean['Skils(★)'] = fifa_clean['Skils(★)'].str.replace('\u2605', '')
fifa_clean['Injury Resistance(★)'] = fifa_clean['Injury Resistance(★)'].str.replace('\u2605', '')

# quick glance at the resulting data
fifa_clean[['Weaker Foot(★)','Skils(★)','Injury Resistance(★)']].sample(5)

,Weaker Foot(★),Skils(★),Injury Resistance(★)
4200,5,4,1
6169,2,3,1
14476,3,2,1
17200,3,2,1
14253,3,3,1


#### Current ages of players from the Age column 
The age column houses ages of players in 2021 when this data was collated to have an updated age value for the year of 2023, a new column would be derived from the previous age column

In [34]:
# add 2 to their previous age to get their current age
fifa_clean['Age(2023)'] = fifa_clean['Age(2021)'] + 2

# a quick glance of samples of the data
fifa_clean[['Age(2021)', 'Age(2023)']].sample(5)

,Age(2021),Age(2023)
9478,33,35
5000,30,32
6109,28,30
14869,23,25
4522,23,25


#### Drop unnecessary columns
The contract and loan date end columns have been formatted already to give newer more imformative and cleaner columns(Contract Type, Contract Start and Contract End) so should be drop, the positions column looks messy and a more unique column(Best Positions) is available to tell us what position the player plays at

In [35]:
# drop the contract, positions and loan date end columns
fifa_clean.drop(['Contract', 'Positions', 'Name', 'Loan Date End'], axis=1, inplace=True)
fifa_clean.columns

Index(['ID', 'Full Name', 'photoUrl', 'playerUrl', 'Nationality', 'Age(2021)',
       'Overall Rating', 'Potential Rating', 'Club', 'Height(ft)',
       'Weight(lbs)', 'Preferred Foot', 'Best Overall Rating', 'Best Position',
       'Joined', 'Value(€)', 'Wage(€)', 'Release Clause(€)', 'Attacking',
       'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
       'Skill', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing',
       'Ball Control', 'Movement', 'Acceleration', 'Sprint Speed', 'Agility',
       'Reactions', 'Balance', 'Power', 'Shot Power', 'Jumping', 'Stamina',
       'Strength', 'Long Shots', 'Mentality', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Penalties', 'Composure', 'Defending',
       'Marking', 'Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Total Stats', 'Base Stats', 'Weaker Foot(★)',
       'Skils(★)', 'Attacking Contribution'

#### The total stats column is an aggregate of the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping Column, each of these columns are also aggregates of other columns which are broken down below:
1.  Attacking Column = Crossing + Finishing + HEading Accuracy + Short Passing + Volleys
2. Skill Column = Dribbling + Curve + FK Accuracy + Long Passing + Ball Control 
3. Movement Column = Acceleration + Sprint Speed + Agility + Reactions + Balance 
4. Power Column = Shot Power + Jumping + Stamina + Strength + Long Shots
5. Mentality Column = Aggression + Interceptions + Positioning + Vision + Penalties
6. Defending Column = Marking + Standing Tackle + Sliding Tackle
7. Goalkeeping = GK Diving + GK Handling + GK Kicking + GK Positioning + GK Reflexes
#### This leaves out the composure column, a quick visit to the sofifa.com site shows this is a little oversight and the composure column was supposed to be added to the Mentality Column and the total affects the total stats.

Other attributes column would be accessed to be sure the calculations done were correct and then the composure column values would be inputted where they ought to bee used for calculations

##### Confirm Attacking column Calculations

In [36]:
# Create a new column "Total Attacking" to store the sum of the attacking attributes
fifa_clean["Total Attacking"] = fifa_clean[["Crossing", "Finishing", "Heading Accuracy", "Short Passing", "Volleys"]].sum(axis=1)

# Check if the "Total Attacking" matches the "Attacking" column for each row
fifa_clean["Attacking Check"] = fifa_clean["Attacking"] == fifa_clean["Total Attacking"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Attacking Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Attacking,Attacking Check


##### drop columns used in comparison

In [37]:
# Drop the "Total Attacking" and "Attacking Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Attacking", "Attacking Check"])

# a quick glance to confirm thy were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
215,217940,Dakonam Djene,https://cdn.sofifa.com/players/217/940/21_60.png,http://sofifa.com/player/217940/dakonam-djene/210006/,Togo,28,82,83,Getafe CF,5.83,158.00,Right,83,CB,2017-07-24,"29,000,000.00","39,000.00","46,800,000.00",263,60,29,77,75,22,289,50,51,48,69,71,366,73,71,71,82,69,327,67,79,74,80,27,305,88,85,49,42,41,73,253,84,86,83,49,7,15,7,10,10,1852,396,4,2,Medium,High,1,72,37,62,61,84,80,88,Permanent,2017,2023,30


##### Confirm Skill column Calculations

In [38]:
# Create a new column "Total Skill" to store the sum of the skill attributes
fifa_clean["Total Skill"] = fifa_clean[['Dribbling','Curve', 'FK Accuracy', 'Long Passing', 'Ball Control']].sum(axis=1)

# Check if the "Total Skill" matches the "Skill" column for each row
fifa_clean["Skill Check"] = fifa_clean["Skill"] == fifa_clean["Total Skill"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Skill Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Skill,Skill Check


##### drop columns used in comparison

In [39]:
# Drop the "Total Skill" and "Skill Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Skill", "Skill Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
6372,200968,Florent Ogier,https://cdn.sofifa.com/players/200/968/21_60.png,http://sofifa.com/player/200968/florent-ogier/210006/,France,31,68,68,Clermont Foot 63,6.00,160.00,Right,68,CB,2018-07-13,"825,000.00","3,000.00","1,200,000.00",191,23,20,71,54,23,221,40,33,34,50,64,272,41,48,58,62,63,335,57,88,86,74,30,240,74,69,30,23,44,62,202,70,69,63,44,14,8,8,8,6,1505,313,3,2,Medium,High,1,45,31,39,51,69,78,1,Permanent,2018,2023,33


##### Confirm Movement column Calculations

In [40]:
# Create a new column "Total Movement" to store the sum of the Movement attributes
fifa_clean["Total Movement"] = fifa_clean[['Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance']].sum(axis=1)

# Check if the "Total Movement" matches the "Movement" column for each row
fifa_clean["Movement Check"] = fifa_clean["Movement"] == fifa_clean["Total Movement"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean['Movement Check'] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Movement,Movement Check


##### drop columns used in comparison

In [41]:
# Drop the "Total Movement" and "Movement Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Movement", "Movement Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
13525,244881,Nair Tiknizyan,https://cdn.sofifa.com/players/244/881/21_60.png,http://sofifa.com/player/244881/nair-tiknizyan/210006/,Russia,21,62,75,PFC CSKA Moscow,5.92,165.00,Left,63,LM,2016-09-30,"975,000.00","6,000.00","1,300,000.00",273,64,55,43,61,50,276,65,61,40,47,63,310,74,73,54,44,65,285,58,63,66,58,40,240,32,40,56,49,63,46,153,51,51,51,45,9,9,5,11,11,1582,347,3,2,High,Medium,1,73,53,56,62,48,55,9,Permanent,2016,2025,23


##### Confirm Power column Calculations

In [42]:
# Create a new column "Total Power" to store the sum of the Power attributes
fifa_clean["Total Power"] = fifa_clean[['Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots']].sum(axis=1)

# Check if the "Total Power" matches the "Power" column for each row
fifa_clean["Power Check"] = fifa_clean["Power"] == fifa_clean["Total Power"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Power Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Power,Power Check


##### drop columns used in comparison

In [43]:
# Drop the "Total Power" and "Power Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Power", "Power Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
14888,234260,Andy Fisher,https://cdn.sofifa.com/players/234/260/21_60.png,http://sofifa.com/player/234260/andy-fisher/210006/,England,22,60,71,Blackburn Rovers,6.00,160.00,Right,60,GK,2016-01-07,"475,000.00","4,000.00","578,000.00",70,12,5,13,33,7,97,13,11,11,28,34,193,36,31,40,45,41,161,46,44,29,37,5,71,21,9,6,24,11,32,51,27,10,14,306,64,59,61,58,64,949,339,3,1,Medium,Medium,1,64,59,61,64,33,58,4,Permanent,2016,2021,24


##### Update Mentality column by adding Composure values to its attributes

In [44]:
# suming  up the  'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties'and add the 'Composure' column to make the Mentality column
fifa_clean['Mentality'] = fifa_clean[[ 'Aggression', 'Interceptions', 'Positioning', 'Vision','Penalties', 'Composure']].sum(axis=1)

# a quick glance to confirm they were dropped
fifa_clean[['Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure']].sample(4)

,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure
4107,385,71,68,67,63,49,67
10258,310,47,24,60,61,54,64
6308,297,60,71,39,32,37,58
18966,216,46,45,25,25,35,40


##### Confirm Defending column Calculations

In [45]:
# Create a new column "Total Defending" to store the sum of the defending attributes
fifa_clean["Total Defending"] = fifa_clean[['Marking', 'Standing Tackle','Sliding Tackle']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Defending Check"] = fifa_clean["Defending"] == fifa_clean["Total Defending"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Defending Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Defending,Defending Check


##### drop columns used in comparison

In [46]:
# Drop the "Total Defending" and "Defending Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Defending", "Defending Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
8579,228632,Marvin Ajani,https://cdn.sofifa.com/players/228/632/21_60.png,http://sofifa.com/player/228632/marvin-ajani/210006/,Germany,26,66,66,SV Wehen Wiesbaden,6.17,189.00,Right,66,RM,2019-07-01,"925,000.00","3,000.00","1,000,000.00",287,67,58,56,62,44,281,64,58,47,52,60,367,81,94,67,62,63,329,67,53,78,75,56,348,73,49,63,55,48,60,161,53,54,54,60,15,16,9,14,6,1773,395,4,3,Medium,Medium,1,88,58,59,63,53,74,7,Permanent,2019,2021,28


##### Confirm Goalkeeping column Calculations

In [47]:
# Create a new column "Total GK" to store the sum of the Goalkeeping attributes
fifa_clean["Total GK"] = fifa_clean[['GK Diving', 'GK Handling','GK Kicking', 'GK Positioning', 'GK Reflexes']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["GK Check"] = fifa_clean["Goalkeeping"] == fifa_clean["Total GK"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["GK Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total GK,GK Check


##### drop columns used in comparison

In [48]:
# Drop the "Total GK" and "GK Check" columns
fifa_clean = fifa_clean.drop(columns=["Total GK", "GK Check"])

# a quick glance to confirm they were dropped
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
12278,188001,Jose Eduardo de Araujo,https://cdn.sofifa.com/players/188/001/21_60.png,http://sofifa.com/player/188001/jose-eduardo-de-araujo/210006/,Brazil,28,63,63,Sandefjord Fotball,6.08,180.00,Right,64,CAM,2020-06-10,"475,000.00","1,000.00","469,000.00",264,57,55,40,69,43,291,64,44,48,68,67,339,70,71,69,65,64,347,63,69,67,84,64,349,70,49,52,61,49,68,138,53,44,41,47,6,7,14,13,7,1707,379,3,3,High,High,1,71,57,63,65,47,76,1,Permanent,2020,2021,30


#### Total stats represent the final value of all the stats a character or object has, which includes their base stats, any bonuses or modifications gained through equipment or leveling up, and any other factors that affect their abilities. Therefore, adding the base stats to the total stats would result in double-counting the base stats. So base stats would be left out in this aggregation

##### Update Total Stats column by with updating values of its attributes

In [49]:
# suming  up the Attacking, Skill, Movement, Power, Mentality, Defending and Goal Keeping to make the Mentality column
fifa_clean['Total Stats'] = fifa_clean[[ 'Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sum(axis=1)

# a view of the resulting total stats column
fifa_clean[[ 'Total Stats','Attacking', 'Skill', 'Movement', 'Power','Mentality', 'Defending']].sample()

,Total Stats,Attacking,Skill,Movement,Power,Mentality,Defending
16552,1600,241,251,278,310,346,174


#### Confirm Base Stats column Calculations
The Base stats refer to the six basic categories of skills used to rate a player's abilities in FIFA. These stats are typically used as a starting point for evaluating a player's performance, and they provide a basic overview of a player's abilities. The six columns would be added to compare if they sum up to the base stats

##### Confirm Base Stats column Calculations

In [50]:
# Create a new column "Total Base Stats" to store the sum of the Base Stats attributes
fifa_clean["Total Base Stats"] = fifa_clean[['Pace', 'Shot',
       'Pass Accuracy', 'Dribbling Ability', 'Defensive Ability',
       'Physicality']].sum(axis=1)

# Check if the "Total Defending" matches the "defending" column for each row
fifa_clean["Base Stats Check"] = fifa_clean["Base Stats"] == fifa_clean["Total Base Stats"]

# check to see if there are values that dont tally
fifa_clean[fifa_clean["Base Stats Check"] == False]

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023),Total Base Stats,Base Stats Check


##### drop columns used in comparison

In [51]:
# Drop the "Total Base Stats" and "Base Check" columns
fifa_clean = fifa_clean.drop(columns=["Total Base Stats", "Base Stats Check"])

# a view of a sample of the data
fifa_clean.sample()

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Profile Views,Contract Type,Contract Start,Contract End,Age(2023)
12806,226759,Ben Kantie Karamoko,https://cdn.sofifa.com/players/226/759/21_60.png,http://sofifa.com/player/226759/ben-kantie-karamoko/210006/,Ivory Coast,25,63,70,Aalesunds FK,6.17,165.00,Right,65,CB,2018-03-21,"875,000.00",950.00,0.00,203,35,22,64,49,33,179,35,29,35,37,43,218,51,40,43,53,31,297,47,77,57,92,24,272,65,57,29,29,39,53,189,62,65,62,47,11,10,8,7,11,1358,291,3,2,Low,Medium,1,45,29,39,39,62,77,1,On Loan,2018,2020,27


#### Reindex Columns

Changing the order of columns can be helpful for organizing the data in a way that makes it easier to understand or analyze.

In [52]:
fifa_clean.insert(loc=7, column='Age(2023)', value=fifa_clean.pop('Age(2023)'))
fifa_clean.insert(loc=8, column='Contract Type', value=fifa_clean.pop('Contract Type'))
fifa_clean.insert(loc=9, column='Contract Start', value=fifa_clean.pop('Contract Start'))
fifa_clean.insert(loc=10, column='Contract End', value=fifa_clean.pop('Contract End'))
fifa_clean.insert(loc=66, column='Pace', value=fifa_clean.pop('Pace'))
fifa_clean.insert(loc=67, column='Shot', value=fifa_clean.pop('Shot'))
fifa_clean.insert(loc=68, column='Pass Accuracy', value=fifa_clean.pop('Pass Accuracy'))
fifa_clean.insert(loc=69, column='Dribbling Ability', value=fifa_clean.pop('Dribbling Ability'))
fifa_clean.insert(loc=70, column='Defensive Ability', value=fifa_clean.pop('Defensive Ability'))
fifa_clean.insert(loc=71, column='Physicality', value=fifa_clean.pop('Physicality'))


fifa_clean.sample(3)

,ID,Full Name,photoUrl,playerUrl,Nationality,Age(2021),Overall Rating,Age(2023),Contract Type,Contract Start,Contract End,Potential Rating,Club,Height(ft),Weight(lbs),Preferred Foot,Best Overall Rating,Best Position,Joined,Value(€),Wage(€),Release Clause(€),Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weaker Foot(★),Pace,Shot,Pass Accuracy,Dribbling Ability,Defensive Ability,Physicality,Skils(★),Attacking Contribution,Defensive Contribution,Injury Resistance(★),Profile Views
795,204024,Christoph Kramer,https://cdn.sofifa.com/players/204/024/21_60.png,http://sofifa.com/player/204024/christoph-kramer/210006/,Germany,29,78,31,Permanent,2016,2023,78,Borussia Monchengladbach,6.25,167.00,Right,78,CDM,2016-07-01,"12,000,000.00","37,000.00","15,300,000.00",302,60,46,61,77,58,324,72,54,51,71,76,300,52,48,64,82,54,308,54,51,90,71,42,422,75,81,70,72,47,77,229,80,77,72,44,8,13,9,6,8,1885,393,3,50,49,69,72,77,76,3,High,High,2,31
3026,204526,Pablo Hervias Rubio,https://cdn.sofifa.com/players/204/526/21_60.png,http://sofifa.com/player/204526/pablo-hervias-rubio/210006/,Spain,27,73,29,Permanent,2019,2022,73,Real Valladolid CF,5.75,147.00,Right,73,RM,2019-07-01,"3,200,000.00","16,000.00","8,800,000.00",298,73,65,40,69,51,369,78,77,75,63,76,381,78,78,80,70,75,309,69,64,71,41,64,335,42,41,65,66,51,70,114,50,31,33,52,7,12,13,12,8,1806,378,3,78,64,69,77,40,50,4,High,Medium,1,9
6714,219995,Vittorio Parigini,https://cdn.sofifa.com/players/219/995/21_60.png,http://sofifa.com/player/219995/vittorio-parigini/210006/,Italy,24,68,26,Permanent,2020,2024,74,Genoa,5.92,154.00,Right,69,ST,2020-02-01,"1,900,000.00","7,000.00","2,400,000.00",318,63,69,56,65,65,299,73,61,47,43,75,355,74,71,76,64,70,313,65,77,55,54,62,299,47,20,68,37,54,73,58,29,18,11,45,12,11,6,10,6,1642,345,3,72,66,55,73,25,54,3,High,Low,1,6


In [53]:
# save cleaned fifa data
fifa_clean.to_csv('Fifa_2021_cleaned.csv',index=False)

# Print a message to confirm the file has been saved
print('fifa_clean.csv saved successfully.')

fifa_clean.csv saved successfully.
